In [1]:
# 流れ
# ・成果シートのデータ全取得
# ・必要なデータ列に絞る
# ・要らない行削る
# ・ASPカラム付け加える
# ・各ASPで上記行い合算する
# ・日付でソート
# ・今月のシートに反映

In [2]:
# 成果確認シートの内容取得

In [3]:
####################
# 
# モジュール
# 
####################

# 一般
import datetime as dt
import pandas as pd

# スプシ関連
from gspread_dataframe import set_with_dataframe

# 自作
import conf
import func

print("モジュールインポート")

モジュールインポート


In [4]:
####################
# 
# 変数
# 
####################

# 今月の成果シート名
today = dt.date.today()
this_year  = str(today.year)
this_month = str(today.month)
sheet_name_seika = this_year + "年" + this_month + "月成果" 

print("変数定義完了")

変数定義完了


In [5]:
####################
# 
# 成果シートの情報取得
# 
####################

# 成果シートにアクセス
key_seika = "1tYvW0ntpbCVOpY2UGcqdQD_DQhA7HesfKkg0NPAp65g" # ブリーチ
wb_seika = func.connect_gspread(key_seika)
ws_seika = wb_seika.worksheet(sheet_name_seika)

# 成果シートのデータ取得
df_seika = pd.DataFrame(ws_seika.get_all_values()) # 元データ取得
df_seika = df_seika.drop(index=df_seika.index[[0,1]]) # 先頭行削除
df_seika = df_seika.reindex(columns=[9,1,2,4,5,6]) # 必要列抽出

# カラム名つける
df_seika = df_seika.rename(columns={
    9:"成果識別番号",
    1:"成果発生日時",
    2:"獲得方法",
    4:"メディア名",
    5:"LP名",
    6:"グロス(税込)"
})

# 日付と時間に列を分ける（レントラだけ仕様が違う）
ach_day  = df_seika['成果発生日時'].str.extract('(\d+[/-]\d+[/-]+\d+)', expand=True) # 成果発生日
ach_day  = ach_day.apply(lambda x: x.str.replace('-','/'))                           # "-"を"/"に変換
ach_time = df_seika['成果発生日時'].str.extract('(\d+:\d:*\d*)', expand=True)        # 成果発生時間

# Macの場合
df_seika.insert(1,"成果発生日",ach_day[0])    # 成果発生日を追加
df_seika.insert(2,"成果発生時間",ach_time[0]) # 成果発生時間を追加

# Windowsの場合
# df_seika.insert(1,"成果発生日",ach_day)    # 成果発生日を追加
# df_seika.insert(2,"成果発生時間",ach_time) # 成果発生時間を追加

# 成果発生日時削除
df_seika = df_seika.drop("成果発生日時",axis=1) 

# ASP名追加
df_seika.insert(5,"ASP","ブリーチ") 

In [6]:
# 該当しない日付を削除
df_seika['成果発生日'] = pd.to_datetime(df_seika['成果発生日'], format='%Y/%m/%d') # 成果発生日の列を日付型に変換
df_seika = df_seika[(df_seika['成果発生日'] >= dt.datetime(2022,1,1)) & (df_seika['成果発生日'] <= dt.datetime(2022,1,31))] # 該当する日付を抽出
df_seika.reset_index(drop=True, inplace=True) # 行番号振り直し